In [1]:
import pandas as pd
df = pd.read_csv('/Users/phuong/Documents/AAAI_Project/All_Articles.csv',encoding ='latin1')
df.head()

,PMID,Article Title,Abstract,Publication Type,Methods Section,First Label,Second Label,hydroxymethylglutaryl,acetaminophen,alfentanyl,...,tramadol,trazodone,triazolam,valproate,venlafaxine,verapamil,voriconazole,warfarin,ziprasidone,zolpidem
0,8646822,A kinetic and dynamic study of oral alprazolam...,To assess the possible involvement of CYP3A4 i...,Clinical Trial ;Journal Article ;Randomized Co...,methods subjects twelve unrelated healthy male...,DDI Clinical Trial,RCT DDI Clinical Trial,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8453848,A potentially hazardous interaction between er...,Interaction between erythromycin and midazolam...,Clinical Trial ;Journal Article ;Randomized Co...,material and methods study design orally admin...,DDI Clinical Trial,RCT DDI Clinical Trial,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23210726,A semi-mechanistic absorption model to evaluat...,The aim of this study was to develop a PK/PD m...,Journal Article ;Randomized Controlled Trial ;...,methods ten healthy male volunteers randomized...,DDI Clinical Trial,RCT DDI Clinical Trial,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11936570,A study of the interaction potential of azithr...,Atorvastatin is a common option among the HMG-...,"Clinical Trial ;Clinical Trial, Phase I ;Compa...",materials and methods this randomized open lab...,DDI Clinical Trial,RCT DDI Clinical Trial,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1387301,Absolute bioavailability of clarithromycin aft...,The absolute bioavailability of clarithromycin...,Clinical Trial ;Comparative Study ;Journal Art...,clarithromycin new 14 membered macrolide antim...,PK Trial,PK Trial,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from io import StringIO

col = ['Article Title','Abstract','Methods Section','First Label']
df = df[col]
df = df[pd.notnull(df['Abstract'])]
print (df)
df.columns = ['Article Title','Abstract','Methods Section','First Label']
df['category_id'] = df['First Label'].factorize()[0]
category_id_df = df[['First Label', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'First Label']].values)
df.head()

                                         Article Title  \
0    A kinetic and dynamic study of oral alprazolam...   
1    A potentially hazardous interaction between er...   
2    A semi-mechanistic absorption model to evaluat...   
3    A study of the interaction potential of azithr...   
4    Absolute bioavailability of clarithromycin aft...   
5    Absolute bioavailability of oral immediate and...   
6    Absolute oral bioavailability of rosuvastatin ...   
7    Amiodarone interacts with simvastatin but not ...   
8    An assessment of drug-drug interactions: the e...   
9    Assessing the clinical significance of botanic...   
10   Atomoxetine hydrochloride: clinical drug-drug ...   
11   Carbamazepine but not valproate induces buprop...   
12   Celecoxib inhibits metabolism of cytochrome P4...   
13   Change from the CYP2D6 extensive metabolizer t...   
14          Cimetidine interaction with amitriptyline.   
15   Cimetidine interaction with imipramine and nor...   
16   Clarithro

,Article Title,Abstract,Methods Section,First Label,category_id
0,A kinetic and dynamic study of oral alprazolam...,To assess the possible involvement of CYP3A4 i...,methods subjects twelve unrelated healthy male...,DDI Clinical Trial,0
1,A potentially hazardous interaction between er...,Interaction between erythromycin and midazolam...,material and methods study design orally admin...,DDI Clinical Trial,0
2,A semi-mechanistic absorption model to evaluat...,The aim of this study was to develop a PK/PD m...,methods ten healthy male volunteers randomized...,DDI Clinical Trial,0
3,A study of the interaction potential of azithr...,Atorvastatin is a common option among the HMG-...,materials and methods this randomized open lab...,DDI Clinical Trial,0
4,Absolute bioavailability of clarithromycin aft...,The absolute bioavailability of clarithromycin...,clarithromycin new 14 membered macrolide antim...,PK Trial,1


In [4]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('First Label').Abstract.count().plot.bar(ylim=0)
plt.show()

/Users/phuong/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


KeyboardInterrupt: 

In [31]:
# SVM Model with bigrams taken from Abstract, Article Title, & Methods Section text
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np 
from sklearn.metrics import confusion_matrix

vectorizer = CountVectorizer(ngram_range=(2,2))
text = df['Abstract'] + df['Article Title'] + df['Methods Section']
features = vectorizer.fit_transform(text)
print (features.shape)
labels = df.category_id

# df_1 = pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names())
# target = df['First Label']
# df_final = pd.concat((df_1,target.rename('Classification_Label')), axis =1)
# df_final.to_csv("/Users/phuong/Documents/AAAI_Project/Features_alltext_test.csv",  encoding='utf-8', index=False)

X_train, X_test, y_train, y_test,  = train_test_split( features, labels, stratify = labels, test_size=0.2, random_state = 0)
model = SVC(kernel='linear', class_weight='balanced')
clf = model.fit(X_train, y_train)
predictions = clf.predict(X_test)

for row_index, (X_test_item, prediction, label) in enumerate(zip (X_test, predictions, labels)):
  if prediction != label:
    print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)

cm = confusion_matrix(y_test, predictions)
print(cm)
tn, fp, fn, tp = cm.ravel()
print (tn, fp, fn, tp)

print ('Accuracy:', accuracy_score(y_test, predictions))
print ('F1 score:', f1_score(y_test, predictions,average='weighted'))
print ('Recall:', recall_score(y_test, predictions,
                              average='weighted'))
print ('Precision:', precision_score(y_test, predictions,
                                    average='weighted'))

print (classification_report(y_test,predictions))

(185, 100113)
Row 4 has been classified as  0 and should be  1
Row 5 has been classified as  0 and should be  1
Row 6 has been classified as  0 and should be  1
Row 13 has been classified as  0 and should be  1
Row 16 has been classified as  0 and should be  1
Row 25 has been classified as  0 and should be  1
Row 26 has been classified as  0 and should be  1
Row 27 has been classified as  0 and should be  1
Row 31 has been classified as  0 and should be  1
Row 32 has been classified as  0 and should be  1
[[28  0]
 [ 8  1]]
28 0 8 1
Accuracy: 0.7837837837837838
F1 score: 0.7108108108108109
Recall: 0.7837837837837838
Precision: 0.8318318318318318
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        28
           1       1.00      0.11      0.20         9

   micro avg       0.78      0.78      0.78        37
   macro avg       0.89      0.56      0.54        37
weighted avg       0.83      0.78      0.71        37



In [32]:
# SVM Model with bigrams taken from Abstract, Article Title, & Methods Section text
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

vectorizer = CountVectorizer(ngram_range=(2,2))
text = df['Abstract'] + df['Article Title'] + df['Methods Section']
features = vectorizer.fit_transform(text)
print (features.shape)
labels = df.category_id
features = features.todense()

X_train, X_test, y_train, y_test,  = train_test_split( features, labels, test_size=0.2, random_state = 0)
model = RandomForestClassifier(random_state=0)
clf = model.fit(X_train, y_train)
predictions = clf.predict(X_test)

for row_index, (X_test_item, prediction, label) in enumerate(zip (X_test, predictions, labels)):
  if prediction != label:
    print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)

cm = confusion_matrix(y_test, predictions)
print(cm)
tn, fp, fn, tp = cm.ravel()
print (tn, fp, fn, tp)

print ('Accuracy:', accuracy_score(y_test, predictions))
print ('F1 score:', f1_score(y_test, predictions,average='weighted'))
print ('Recall:', recall_score(y_test, predictions,
                              average='weighted'))
print ('Precision:', precision_score(y_test, predictions,
                                    average='weighted'))

print (classification_report(y_test,predictions))


(185, 100113)
Row 4 has been classified as  0 and should be  1
Row 5 has been classified as  0 and should be  1
Row 6 has been classified as  0 and should be  1
Row 13 has been classified as  0 and should be  1
Row 16 has been classified as  0 and should be  1
Row 22 has been classified as  0 and should be  1
Row 25 has been classified as  0 and should be  1
Row 26 has been classified as  0 and should be  1
Row 27 has been classified as  0 and should be  1
Row 31 has been classified as  0 and should be  1
Row 32 has been classified as  0 and should be  1
[[28  0]
 [ 9  0]]
28 0 9 0
Accuracy: 0.7567567567567568
F1 score: 0.651975051975052
Recall: 0.7567567567567568
Precision: 0.5726807888970051
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        28
           1       0.00      0.00      0.00         9

   micro avg       0.76      0.76      0.76        37
   macro avg       0.38      0.50      0.43        37
weighted avg       0.57  

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Run classifier with different algorithms 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.utils import class_weight

models = [
    RandomForestClassifier(random_state=0, class_weight = 'balanced'),
    SVC(kernel='linear', class_weight='balanced'),
    LogisticRegression(random_state=0, class_weight = 'balanced')
]

vectorizer = CountVectorizer(ngram_range=(2,2))
text = df['Abstract'] + df['Article Title'] + df['Methods Section']
features_2 = vectorizer.fit_transform(text)
features_2 = features_2.todense()

labels = df.category_id

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features_2, labels, scoring='accuracy', cv=CV)
    auc_scores = cross_val_score(model, features_2, labels, scoring='roc_auc', cv=CV)
    precisions = cross_val_score(model, features_2, labels, scoring='precision', cv=CV)
    recalls = cross_val_score(model, features_2, labels, scoring='recall', cv=CV) 
    f1_scores = cross_val_score(model, features_2, labels, scoring='f1', cv=CV)
    
    print ('acc\t%.4f\t%s' % (np.mean(accuracies), model_name))
    print ('auc\t%.4f\t%s' % (np.mean(auc_scores), model_name))
    print ('prec\t%.4f\t%s' % (np.mean(precisions), model_name))
    print ('recall\t%.4f\t%s' % (np.mean(recalls), model_name))
    print ('f1\t%.4f\t%s' % (np.mean(f1_scores), model_name))

acc	0.7675	RandomForestClassifier
auc	0.7436	RandomForestClassifier
prec	0.5889	RandomForestClassifier
recall	0.2000	RandomForestClassifier
f1	0.2653	RandomForestClassifier


In [6]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.2.2.
The scikit-learn version is 0.15.2.
